#**SQuAD 2.0 with sentence-level BERT model**

---



## **1. Mount Google Drive**

In [1]:
# Mount Google Drive.
# Copy the authentication code into the input box.
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
BASE_DATA_DIR = "/content/drive/My\ Drive/Colab\ Notebooks/w266_project"
SQUAD_DATA_DIR = BASE_DATA_DIR + "/squad_2"
BERT_DATA_DIR = BASE_DATA_DIR + "/bert"  
# Model output on Google Drive
DRIVE_OUTPUT_DIR = BASE_DATA_DIR + "/model_output"

In [3]:
cd $BASE_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project


In [0]:
ls -l -R

.:
total 1310001
drwx------ 2 root root       4096 Mar  6 07:48 bert/
drwx------ 2 root root       4096 Mar 22 00:35 bert_repo/
drwx------ 2 root root       4096 Mar 22 00:35 classification/
-rw------- 1 root root    1014050 Mar 23 20:07 dev_context_prediction_0_0_0.1.csv
-rw------- 1 root root    5963897 Mar 23 06:58 dev_sentence_prediction_0_0_0.0001.csv
-rw------- 1 root root    5963971 Mar 23 07:40 dev_sentence_prediction_0_0_0.001.csv
-rw------- 1 root root    5963971 Mar 23 08:11 dev_sentence_prediction_0_0_0.005.csv
-rw------- 1 root root    5963971 Mar 23 08:23 dev_sentence_prediction_0_0_0.008.csv
-rw------- 1 root root    5963971 Mar 23 17:30 dev_sentence_prediction_0_0_0.1.csv
-rw------- 1 root root         71 Mar 23 05:40 dev_sentence_prediction_0_1.csv
-rw------- 1 root root     967145 Mar 23 05:42 dev_sentence_prediction_10000_20000.csv
-rw------- 1 root root     871473 Mar 23 05:44 dev_sentence_prediction_20000_30000.csv
-rw------- 1 root root     973642 Mar 23 05:45 dev

## **2. Download SQuAd 2.0 Dataset**


### 2.1 Download and Sanity Check the data

In [0]:
cd $SQUAD_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/squad_2


In [0]:
#Download the SQUAD train and dev dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-04-08 05:12:18--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.3’

train-v2.0.json.3   100%[===================>]  40.17M  49.6MB/s    in 0.8s    

2020-04-08 05:12:19 (49.6 MB/s) - ‘train-v2.0.json.3’ saved [42123633/42123633]

--2020-04-08 05:12:20--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json.3’

dev-v2.0.json.

In [0]:
# Sanity check the files 
import json

def GetStats(filename):
    with open(filename) as f:
        data = json.load(f)
    st_articles = 0
    st_paragraphs = 0
    st_questions = 0
    st_questions_with_answer = 0 
    st_questions_no_answer = 0
    st_paragraphs_no_answer = 0

    for article in data['data']:
        st_articles += 1 
        for paragraph in article['paragraphs']:
            qas = paragraph['qas']
            #for qa in qas:
            #    if len(qa['answers']) != 1 :
            #        print("Wwarning: ", str(qa))

            new_qas = [x for x in qas if not x['is_impossible']]   
            if len(new_qas) == 0:
                st_paragraphs_no_answer += 1 
            st_paragraphs += 1
            st_questions += len(qas)
            st_questions_with_answer += len(new_qas)
            st_questions_no_answer += (len(qas) - len(new_qas))  
    print("Filename: %s, articles: %d, paragraphs: %d, questions: %d, questions_with_answer:%d, questons_no_answer: %d, "
           "paragraphs without answerable questions: %d" %
          (filename, st_articles, st_paragraphs, st_questions, st_questions_with_answer, st_questions_no_answer,
            st_paragraphs_no_answer))

In [6]:
GetStats("./dev-v2.0.json")
GetStats("./train-v2.0.json")

Filename: ./dev-v2.0.json, articles: 35, paragraphs: 1204, questions: 11873, questions_with_answer:5928, questons_no_answer: 5945, paragraphs without answerable questions: 0
Filename: ./train-v2.0.json, articles: 442, paragraphs: 19035, questions: 130319, questions_with_answer:86821, questons_no_answer: 43498, paragraphs without answerable questions: 153


### 2.2 Convert TRAIN and DEV sets into TSV format

In [0]:
import json
import copy

def ConvertToTsv(input_file, output_file):
    with open(input_file) as f:
        data = json.load(f)
    out_f = open(output_file, "w") 

    for article in data['data']: 
        for pa in article['paragraphs']:
            context = pa['context']
            context = context.replace("\t", " ")
            context = context.replace("\r", " ")
            context = context.replace("\n", " ")
            for q in pa['qas']:
                question = q['question']
                question = question.replace("\t", " ")
                question = question.replace("\r", " ")
                question = question.replace("\n", " ")
                if q['is_impossible']:
                    out_f.write("0" + "\t" + q['id'] + "\t" + 'context_id' + "\t" + question + "\t" + context + "\n")
                else:
                    out_f.write("1" + "\t" + q['id'] + "\t" + 'context_id' + "\t" + question + "\t" + context + "\n") 
    out_f.close()
    print("Input file: %s, output_file: %s" % (input_file, output_file))
    

In [8]:
ConvertToTsv("./dev-v2.0.json", "./dev_context.tsv")
ConvertToTsv("./train-v2.0.json", "./train_context.tsv") 

Input file: ./dev-v2.0.json, output_file: ./dev_context.tsv
Input file: ./train-v2.0.json, output_file: ./train_context.tsv


In [0]:
!wc -l ./dev_context.tsv
# HasAns questions
!grep "^1" ./dev_context.tsv | wc -l
# NoAns questions
!grep "^0" ./dev_context.tsv | wc -l
# Check some sample
!head ./dev_context.tsv

11873 ./dev_context.tsv
5928
5945
1	56ddde6b9a695914005b9628	context_id	In what country is Normandy located?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
1	56ddde6b9a695914005b9629	context_id	When were the Normans in Normandy?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the

In [0]:
!wc -l ./train_context.tsv
# HasAns questions
!grep "^1" ./train_context.tsv | wc -l
# NoAns questions
!grep "^0" ./train_context.tsv | wc -l
# Check some sample
!head ./dev_context.tsv

130319 ./train_context.tsv
86821
43498
1	56ddde6b9a695914005b9628	context_id	In what country is Normandy located?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
1	56ddde6b9a695914005b9629	context_id	When were the Normans in Normandy?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) wer

## **3. Convert SQuAD 2.0 Data into Sentence Level**


### **3.1 Structure of SQuAD data**

* There is a boolean field 'is_impossible' indicating the question is answerable or not.
* The data contains many articles, each article contains many paragraphs, each paragraph contains many Q&A-s and the context.

```
{
  "version": "v2.0",
  "data": [
    {
      "title": "Normans",
      "paragraphs": [
        {
          "qas": [
            {
              "question": "In what country is Normandy located?",
              "id": "56ddde6b9a695914005b9628",
              "answers": [
                {
                  "text": "France",
                  "answer_start": 159
                },
                {
                  "text": "France",
                  "answer_start": 159
                },
                {
                  "text": "France",
                  "answer_start": 159
                },
                {
                  "text": "France",
                  "answer_start": 159
                }
              ],
              "is_impossible": false
            },  
            {
              "plausible_answers": [
                {
                  "text": "Normans",
                  "answer_start": 4
                }
              ],
              "question": "Who gave their name to Normandy in the 1000's and 1100's",
              "id": "5ad39d53604f3c001a3fe8d1",
              "answers": [
                
              ],
              "is_impossible": true
            },
            ... OTHER QUESTION & ANSWERS ...
          ]  # END OF 'qas'
          "context": "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
        },
        ... OTHER PARAGRAPHS ...
      ]  # END of 'paragraphs'
    },
    ... OTHER ARTICLES ...
  ]  # END OF 'data'
} 
```



### **3.2 Break down TRAIN set to sentence level**

In [10]:
#############################################
# Please note that:
# 1. We filter out small sentences so that the train set won't be too large
# 2. We use 200 characters as threshold for NoAns questions, and 50 characters
#    for HasAns questions, the thresholds are tuned in a way that NoAns and HasAns
#    questions are roughly half and half in training set, without this, NoAns 
#    question will dominate the train-set.
#############################################
import json
import copy

with open("./train-v2.0.json") as f:
    train = json.load(f)

train_new = copy.deepcopy(train) 

# Clear paragraphs
for article in train_new['data']: 
    pa = article['paragraphs']
    pa.clear() 

def CreatePa(is_impossible, sentence, question, question_id, answer_start, answer):  
    pa_template = '''{
          "qas": [
            {
              "question": "",
              "id": "",
              "answers": [
                {
                  "text": "",
                  "answer_start": -1
                }
              ],
              "is_impossible": false
            }
          ], 
          "context": ""
         }'''  
    c_pa = json.loads(pa_template)
    c_pa['qas'][0]['is_impossible'] = is_impossible
    c_pa['context'] = sentence
    c_pa['qas'][0]['question'] = question
    c_pa['qas'][0]['id'] = question_id
    c_pa['qas'][0]['answers'].clear()
    if not is_impossible: 
        for i in range(len(answer)):
            anw = {}
            anw['text'] = answer[i]
            anw['answer_start'] = answer_start[i]
            c_pa['qas'][0]['answers'].append(anw) 

    return c_pa 

questions_added = 0
questions_filtered = 0
has_answers = 0
no_answers = 0

# Also output to TSV format to train classifier model.
train_tsv_sec_f = open("train_sentence_json_190k.tsv", 'w') 

for i in range(len(train['data'])):
    article = train['data'][i]
    new_article = train_new['data'][i] 
    new_paragraphs = new_article['paragraphs'] # Empty

    for pa in article['paragraphs']:
        sentences = pa['context'].split('. ') 
        end = 0
        for sec in sentences:
            sec = sec + '. '
            sec = sec.replace("\t", " ")
            sec = sec.replace("\r", " ")
            sec = sec.replace("\n", " ")
            end += len(sec) 
            for ques in pa['qas']:
                question = ques['question']
                question = question.replace("\t", " ")
                question = question.replace("\r", " ")
                question = question.replace("\n", " ")

                question_id = ques['id'] + "_" + str(end)
                is_impossible = ques['is_impossible'] 
                if is_impossible: 
                    # For NoAns questions, do not output if the sentence is shorter than 200 characters
                    if len(sec) > 200 and len(question) > 1:
                        new_pa = CreatePa(True, sec, question, question_id, [-1], [""])
                        new_paragraphs.append(new_pa)
                        questions_added += 1
                        no_answers += 1 
                        train_tsv_sec_f.write("0" + "\t" + question_id + "\t" + "content-id" + "\t" + question +  "\t" + sec + "\n")
                    else:
                        questions_filtered += 1
                else:    
                    if len(ques['answers']) != 1:
                        print("Warning, answer not equal to 1 in train")
                    potential_answer = ques['answers'][0]['text']
                    answer_start = ques['answers'][0]['answer_start']
                    # For HasAns questions, do not output if the sentence is shorter than 50 characters 
                    if answer_start + len(potential_answer) < end and answer_start >= end - len(sec):
                        if len(sec) > 50 and len(question) > 1:
                            new_pa = CreatePa(False, sec + "...", question, question_id, [answer_start - (end-len(sec))], [potential_answer])
                            new_paragraphs.append(new_pa)
                            questions_added += 1 
                            has_answers += 1 
                            train_tsv_sec_f.write("1" + "\t" + question_id + "\t" + "content-id" + "\t" + question + "\t" + sec +  "\n") 
                        else:
                            questions_filtered    
                    else:
                        # For NoAns questions, do not output if the sentence is shorter than 200 characters 
                        if len(sec) > 200 and len(question) > 1:
                            new_pa = CreatePa(True, sec, question, question_id, [-1], [""])
                            new_paragraphs.append(new_pa)
                            no_answers += 1
                            questions_added += 1
                            train_tsv_sec_f.write("0" + "\t" + question_id + "\t" + "content-id" + "\t" + question + "\t" + sec + "\n") 
                        else:
                            questions_filtered += 1

print("Added: %d, Filtered: %d, Has-answer: %d, No-answer: %d" % (questions_added, questions_filtered, has_answers, no_answers))

train_tsv_sec_f.close() 
with open("train_sentence_json_190k.json", 'w') as f:
    f.write(json.dumps(train_new))   

Added: 191141, Filtered: 475131, Has-answer: 84546, No-answer: 106595


In [11]:
GetStats("./train_sentence_json_190k.json")
#Filename: ./train_sentence_json_190k.json, articles: 442, paragraphs: 191141, questions: 191141, questions_with_answer:84546, questons_no_answer: 106595, paragraphs without answerable questions: 106595

GetStats("./train_sentence_json_3_1_130k.json")


Filename: ./train_sentence_json_190k.json, articles: 442, paragraphs: 191141, questions: 191141, questions_with_answer:84546, questons_no_answer: 106595, paragraphs without answerable questions: 106595
Filename: ./train_sentence_json_3_1_130k.json, articles: 442, paragraphs: 191141, questions: 191141, questions_with_answer:84546, questons_no_answer: 106595, paragraphs without answerable questions: 106595


In [0]:
!head ./train_sentence_json_190k.tsv

1	56be85543aeaaa14008c9063_337	content-id	When did Beyonce start becoming popular?	Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. 
1	56be85543aeaaa14008c9065_337	content-id	What areas did Beyonce compete in when she was growing up?	Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. 
1	56bf6b0f3aeaaa14008c9601_337	content-id	In what city and state did Beyonce  grow up? 	Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. 
1	56bf6b0f3aeaaa14008c9602_337	content-id	In which decade did Beyonce become famous?	Born and raised in Houston, Texas, she performed in various singing an

### **3.3 Break down DEV set to sentence level**

In [12]:
###############################################
# NOTE:
# - The major difference from TRAIN set breakdown is we use the threshold
#   (50-charactoers) to filter out short sentences, no matter the question is 
#   NoAns or HasAns.
# - Another difference is, it may have 
###############################################

# Extract answerable questions for dev set. 
import json
import copy

with open("./dev-v2.0.json") as f:
    dev = json.load(f)

dev_new = copy.deepcopy(dev) 

# Clear paragraphs
for article in dev_new['data']: 
    pa = article['paragraphs']
    pa.clear() 

questions_added = 0
questions_filtered = 0
has_answers = 0
no_answers = 0

# Also output TSV format for debugging purpose
dev_tsv_f = open("./dev_sentence_json.tsv", 'w')

for i in range(len(dev['data'])):
    article = dev['data'][i]
    new_article = dev_new['data'][i] 
    new_paragraphs = new_article['paragraphs'] # Empty

    for pa in article['paragraphs']:
        sentences = pa['context'].split('. ') 
        end = 0
        for sec in sentences:
            sec = sec + '. '
            sec = sec.replace("\t", " ")
            sec = sec.replace("\r", " ")
            sec = sec.replace("\n", " ")
            end += len(sec) 
            for ques in pa['qas']:
                question = ques['question']
                question = question.replace("\t", " ")
                question = question.replace("\r", " ")
                question = question.replace("\n", " ")
                question_id = ques['id'] + "_" + str(end)
                is_impossible = ques['is_impossible']

                if is_impossible: 
                    if len(sec) > 50 and len(question) > 1:
                        new_pa = CreatePa(True, sec, question, question_id, [-1], [""])
                        new_paragraphs.append(new_pa)
                        questions_added += 1
                        no_answers += 1
                        dev_tsv_f.write("0" + "\t" + question_id + "\t" + "content-id" + "\t" + question + "\t" + sec + "\t" + potential_answer + "\n") 
                    else:
                        questions_filtered += 1    
                else:    
                    answer_start = []
                    answers = [] 

                    for an in ques['answers']:  
                        tmp_an = an['text']
                        tmp_start = an['answer_start']
                        if tmp_start + len(tmp_an) < end and tmp_start >= end - len(sec) and len(sec) > 50 and len(question) > 1:
                            answer_start.append(tmp_start - (end-len(sec)))
                            answers.append(tmp_an)
                    
                    if len(answer_start) > 0:
                        new_pa = CreatePa(False, sec + "...", question, question_id, answer_start, answers)
                        new_paragraphs.append(new_pa)
                        questions_added += 1
                        has_answers += 1
                        dev_tsv_f.write("1" + "\t" + question_id + "\t" + "content-id" + "\t" + question + "\t" + sec + "\t" + str(answers) + "\n")  
                    else: 
                        if len(sec) > 50 and len(question) > 1:
                            new_pa = CreatePa(True, sec, question, question_id, [-1], [""])
                            new_paragraphs.append(new_pa)
                            questions_added += 1
                            no_answers += 1 
                            dev_tsv_f.write("0" + "\t" + question_id + "\t" + "content-id" + "\t" + question + "\t" + sec + "\t" + str(answers) + "\n") 
                        else:
                            questions_filtered += 1     

print("Added: %d, filtered: %d, has_answer: %d, no_anser: %d" %
      (questions_added, questions_filtered, has_answers, no_answers))
dev_tsv_f.close()
with open("./dev_sentence_json.json", 'w') as f:
    f.write(json.dumps(dev_new))  

#Added: 61225, filtered: 3375, has_answer: 5782, no_anser: 55443   

Added: 61225, filtered: 3375, has_answer: 6291, no_anser: 54934


In [13]:
GetStats("./dev_sentence_json.json")

Filename: ./dev_sentence_json.json, articles: 35, paragraphs: 61225, questions: 61225, questions_with_answer:6291, questons_no_answer: 54934, paragraphs without answerable questions: 54934


In [15]:
!head ./dev_sentence_json.tsv

1	56ddde6b9a695914005b9628_167	content-id	In what country is Normandy located?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. 	['France', 'France', 'France', 'France']
1	56ddde6b9a695914005b9629_167	content-id	When were the Normans in Normandy?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. 	['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries']
0	56ddde6b9a695914005b962a_167	content-id	From which countries did the Norse originate?	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. 	[]
0	56ddde6b9a695914005b962b_167	content-id	Who was the Norse leader?	The Normans (Norman: Nou

## **4. Download Bert Tools and Data**


### **4.1 Clone the BERT github repository**

In [0]:
# Switch to BERT data dir first.
cd $BERT_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/bert


In [0]:
# Only need to run for 1st time.
!git clone https://github.com/google-research/bert.git

In [0]:
ls -l

total 3701430
drwx------ 3 root root       4096 Mar 24 06:24 bert/
-rw------- 1 root root       1323 Mar  6 07:48 CONTRIBUTING.md
-rw------- 1 root root      16475 Mar  6 07:48 create_pretraining_data.py
-rw------- 1 root root    4370528 Feb 26 06:16 dev-v2.0.json
-rw------- 1 root root      13898 Mar  6 07:48 extract_features.py
-rw------- 1 root root        616 Mar  6 07:48 __init__.py
-rw------- 1 root root       1782 Mar  6 20:01 input_file.json
-rw------- 1 root root      11358 Mar  6 07:48 LICENSE
-rw------- 1 root root      37922 Mar  6 07:48 modeling.py
-rw------- 1 root root       9191 Mar  6 07:48 modeling_test.py
-rw------- 1 root root      11242 Mar  6 07:48 multilingual.md
-rw------- 1 root root       6258 Mar  6 07:48 optimization.py
-rw------- 1 root root       1721 Mar  6 07:48 optimization_test.py
drwx------ 2 root root       4096 Mar  6 20:01 output/
-rw------- 1 root root      66488 Mar  6 07:48 predicting_movie_reviews_with_bert_on_tf_hub.ipynb
drwx------ 2 root roo

### **4.2 Download BERT pre-trained MODEL**


BERT Pretrained Model List :


*   [BERT-Large, Uncased (Whole Word Masking)](https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip) : 24-layer, 1024-hidden, 16-heads, 340M parameters
*   [BERT-Large, Cased (Whole Word Masking)](https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip) : 24-layer, 1024-hidden, 16-heads, 340M parameters
*   [BERT-Base, Uncased](https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip) : 12-layer, 768-hidden, 12-heads, 110M parameters
*   [BERT-Large, Uncased](https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip) : 24-layer, 1024-hidden, 16-heads, 340M parameters
*   [BERT-Base, Cased](https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip): 12-layer, 768-hidden, 12-heads , 110M parameters
*   [BERT-Large, Cased](https://storage.googleapis.com/bert_models/2018_10_18/cased_L-24_H-1024_A-16.zip) : 24-layer, 1024-hidden, 16-heads, 340M parameters
*   [BERT-Base, Multilingual Cased (New, recommended)](https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip) : 104 languages, 12-layer, 768-hidden, 12-heads, 110M parameters
*   [BERT-Base, Multilingual Uncased (Orig, not recommended) (Not recommended, use Multilingual Cased instead)](https://storage.googleapis.com/bert_models/2018_11_03/multilingual_L-12_H-768_A-12.zip) : 102 languages, 12-layer, 768-hidden, 12-heads, 110M parameters
*   [BERT-Base, Chinese](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip) : Chinese Simplified and Traditional, 12-layer, 768-hidden, 12-heads, 110M parameters

BERT has release **BERT-Base** and **BERT-Large** models. Uncased means that the text has been lowercased before WordPiece tokenization, e.g., John Smith becomes john smith, whereas Cased means that the true case and accent markers are preserved. 

**When using a cased model, make sure to pass --do_lower=False at the time of training.** 

You can download any model of your choice. We have used **BERT-Large-Uncased Model.**


In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip

--2020-03-29 04:06:38--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip.3’

uncased_L-24_H-1024 100%[===================>]   1.16G  62.4MB/s    in 18s     

2020-03-29 04:06:56 (67.0 MB/s) - ‘uncased_L-24_H-1024_A-16.zip.3’ saved [1247797031/1247797031]



In [0]:
!unzip uncased_L-24_H-1024_A-16.zip

### **4.3 Move Pretrained Model to GCS Bucket**

In [16]:
%tensorflow_version 1.x
# Create output directory on GCS Bucket

import tensorflow as tf

#####################################################
BUCKET = 'bk_266_pro' #@param {type:"string"}
#####################################################

from google.colab import auth
auth.authenticate_user()

assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'bert_output_190k_sentence' #@param {type:"string"}

BUCKET_NAME = 'gs://{}'.format(BUCKET) 
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, output_dir_name) 
tf.gfile.MakeDirs(OUTPUT_DIR) 

print('***** Model output directory: {} *****'.format(OUTPUT_DIR)) 
print('***** BUCKET_NAME: {} *****'.format(BUCKET_NAME))

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

***** Model output directory: gs://bk_266_pro/bert_output_190k_sentence *****
***** BUCKET_NAME: gs://bk_266_pro *****


In [0]:
!gsutil cp -r $BERT_DATA_DIR/uncased_L-24_H-1024_A-16 $BUCKET_NAME

Copying file:///content/drive/My Drive/Colab Notebooks/w266_project/bert/uncased_L-24_H-1024_A-16/bert_config.json [Content-Type=application/json]...
Copying file:///content/drive/My Drive/Colab Notebooks/w266_project/bert/uncased_L-24_H-1024_A-16/vocab.txt [Content-Type=text/plain]...
Copying file:///content/drive/My Drive/Colab Notebooks/w266_project/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Copying file:///content/drive/My Drive/Colab Notebooks/w266_project/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage

## **5. Setup TPU environment**
*   Verify that you are connected to a TPU device
*   You will get know your TPU Address that is used at time of fine-tuning
*   Perform Google Authentication to access your bucket
*   Upload your credentials to TPU to access your GCS bucket

In [0]:
%tensorflow_version 1.x
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf


assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.1.177.186:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12840125069293181511),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13281360546743657870),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1876384020706440920),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14732964146380544194),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6084337184168705955),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 17478107863603742633),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 13312865584703064038),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 3821180805320265986),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 8003649

## **6. Train The Model**

> Below is the command to run the training. To run the training on TPU you need to make sure about below Hyperparameter, that is tpu must be true and provide the tpu_address that we have find out above.

1.   --use_tpu=True
2.   --tpu_name=YOUR_TPU_ADDRESS





In [0]:
TRAIN_DATA_ALL = SQUAD_DATA_DIR + "/train_sentence_json_190k.json"  #"/train-v2.0.json"
#TRAIN_DATA_ALL = SQUAD_DATA_DIR + "/train_sentence_json_3_1_130k.json"  #"/train-v2.0.json"

DEV_DATA_ALL = SQUAD_DATA_DIR + "/dev_sentence_json.json"  #"/dev-v2.0.json" 

In [0]:
!md5sum $DEV_DATA_ALL

4d41d7d0fc721eed234c600c29b36951  /content/drive/My Drive/Colab Notebooks/w266_project/squad_2/dev_sentence_json.json


In [0]:
# Kick off the training, may take a long time depending on size of train data.
!python $BERT_DATA_DIR/run_squad.py \
  --tpu_name=$TPU_ADDRESS \
  --version_2_with_negative=True \
  --output_dir=$OUTPUT_DIR \
  --train_file=$TRAIN_DATA_ALL \
  --predict_file=$DEV_DATA_ALL \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint="gs://bk_266_pro/bert_output_190k_sentence/model.ckpt-15928" \
  --do_train=False \
  --do_predict=True \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --use_tpu=True  

#$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_model.ckpt \

Streaming output truncated to the last 5000 lines.
I0412 05:10:21.693980 140246288394112 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0412 05:10:21.694124 140246288394112 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0412 05:10:21.711132 140246288394112 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0412 05:10:21.711286 140246288394112 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0412 05:10:21.728137 140246288394112 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0412 05:10:21.728262 140246288394112 tpu_estimator.py:604] Dequeue next (1) batch(es) of data

## **7. Test run the model**



In [0]:
cd $BASE_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project


In [0]:
!touch ./model_output/input_file.json

In [0]:
%%writefile ./model_output/input_file.json
{
    "version": "v2.0",
    "data": [
        {
            "title": "your_title",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "Who is current CEO?",
                            "id": "56ddde6b9a695914005b9628",
                            "is_impossible": ""
                        },
                        {
                            "question": "Who founded google?",
                            "id": "56ddde6b9a695914005b9629",
                            "is_impossible": ""
                        },
                        {
                            "question": "when did IPO take place?",
                            "id": "56ddde6b9a695914005b962a",
                            "is_impossible": ""
                        }
                    ],
                    "context": "Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock. They incorporated Google as a privately held company on September 4, 1998. An initial public offering (IPO) took place on August 19, 2004, and Google moved to its headquarters in Mountain View, California, nicknamed the Googleplex. In August 2015, Google announced plans to reorganize its various interests as a conglomerate called Alphabet Inc. Google is Alphabet's leading subsidiary and will continue to be the umbrella company for Alphabet's Internet interests. Sundar Pichai was appointed CEO of Google, replacing Larry Page who became the CEO of Alphabet."                
                 }
            ]
        }
    ]
}

Overwriting ./model_output/input_file.json


In [0]:
ls

bert/
bert_repo/
classification/
dev_context_prediction_0_0_0.1.csv
dev_sentence_prediction_0_0_0.0001.csv
dev_sentence_prediction_0_0_0.001.csv
dev_sentence_prediction_0_0_0.005.csv
dev_sentence_prediction_0_0_0.008.csv
dev_sentence_prediction_0_0_0.1.csv
dev_sentence_prediction_0_1.csv
dev_sentence_prediction_10000_20000.csv
dev_sentence_prediction_20000_30000.csv
dev_sentence_prediction_30000_40000.csv
dev_sentence_prediction_40000_50000.csv
dev_sentence_prediction_50000_60000.csv
dev_sentence_prediction_60000_0.csv
dev_sentence_prediction.csv
dev_s_prediction_0_0_0.5.csv
dev-v2.0.json
input_file.json
model_output/
squad_2/
train-v2.0.json
uncased_L-24_H-1024_A-16/
uncased_L-24_H-1024_A-16.zip


**Prediction**



In [0]:
!python ./bert/run_squad.py \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=$OUTPUT_DIR/model.ckpt-15928 \
  --do_train=False \
  --max_query_length=384  \
  --do_predict=True \
  --predict_file=./model_output/input_file.json \
  --predict_batch_size=8 \
  --n_best_size=3 \
  --max_seq_length=400 \
  --doc_stride=128 \
  --output_dir=./model_output

In [0]:
cat ./model_output/predictions.json

{
    "56ddde6b9a695914005b9628": "Sundar Pichai",
    "56ddde6b9a695914005b9629": "Larry Page and Sergey Brin",
    "56ddde6b9a695914005b962a": "August 19, 2004"
}


In [0]:
cat ./model_output/nbest_predictions.json

{
    "56ddde6b9a695914005b9628": [
        {
            "text": "Sundar Pichai",
            "probability": 0.5857520515880563,
            "start_logit": -6.278100967407227,
            "end_logit": -7.002716541290283
        },
        {
            "text": "Sundar Pichai was appointed CEO of Google, replacing Larry Page",
            "probability": 0.3442985816361685,
            "start_logit": -6.278100967407227,
            "end_logit": -7.534103870391846
        },
        {
            "text": "Larry Page",
            "probability": 0.06994936677577518,
            "start_logit": -7.871838569641113,
            "end_logit": -7.534103870391846
        }
    ],
    "56ddde6b9a695914005b9629": [
        {
            "text": "Larry Page and Sergey Brin",
            "probability": 0.9367839938386602,
            "start_logit": -5.660099983215332,
            "end_logit": -7.500235080718994
        },
        {
            "text": "Google was founded in 1998 by Larry Page and Ser

## **8. Model Evaluation**


### **8.1 Download SQuAD Eval Script** 

In [0]:
cd $SQUAD_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/squad_2


In [0]:
#download evaluation scripts
!wget https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/

--2020-03-29 23:37:23--  https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/
Resolving worksheets.codalab.org (worksheets.codalab.org)... 40.71.231.153
Connecting to worksheets.codalab.org (worksheets.codalab.org)|40.71.231.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: codalab_session=""; expires=Thu, 01 Jan 1970 00:00:00 GMT; Max-Age=-1; Path=/ at position 70.
Length: unspecified [text/x-python]
Saving to: ‘index.html’

index.html              [ <=>                ]  10.30K  --.-KB/s    in 0.001s  

2020-03-29 23:37:23 (15.9 MB/s) - ‘index.html’ saved [10547]



In [0]:
 mv  index.html evaluate-v2.0.py

### **8.2 Sentence Level Evaluation**

In [0]:
!gsutil cp $OUTPUT_DIR/nbest_predictions.json $DRIVE_OUTPUT_DIR/sentence_nbest_predictions.json

CommandException: No URLs matched: gs://bk_266/bert_output_190k_sentence/nbest_predictions.json


In [0]:
!gsutil cp $OUTPUT_DIR/predictions.json $DRIVE_OUTPUT_DIR/sentence_predictions.json

Copying gs://bk_266_pro/bert_output_3_1_130k_sentence/predictions.json...
/ [1 files][  2.5 MiB/  2.5 MiB]                                                
Operation completed over 1 objects/2.5 MiB.                                      


In [0]:
!head  $DRIVE_OUTPUT_DIR/sentence_predictions.json

{
    "56ddde6b9a695914005b9628_167": "France",
    "56ddde6b9a695914005b9629_167": "10th and 11th centuries",
    "56ddde6b9a695914005b962a_167": "",
    "56ddde6b9a695914005b962b_167": "",
    "56ddde6b9a695914005b962c_167": "",
    "5ad39d53604f3c001a3fe8d1_167": "",
    "5ad39d53604f3c001a3fe8d2_167": "",
    "5ad39d53604f3c001a3fe8d3_167": "",
    "5ad39d53604f3c001a3fe8d4_167": "",


In [0]:
!python ./evaluate-v2.0.py  ./dev_sentence_json.json $DRIVE_OUTPUT_DIR/sentence_predictions.json

{
  "exact": 97.95345038791343,
  "f1": 98.79693432098996,
  "total": 61225,
  "HasAns_exact": 81.37021141312987,
  "HasAns_f1": 89.57912951877367,
  "HasAns_total": 6291,
  "NoAns_exact": 99.85255033312703,
  "NoAns_f1": 99.85255033312703,
  "NoAns_total": 54934
}


In [0]:
# For testing / debugging purpose only.
import json
with open("../model_output/sentence_predictions.json") as f:
    pred = json.load(f)

with open("./dev_sentence_json.json") as f:
    dev = json.load(f)

no_answer = 0
no_answer_correct = 0
has_answer = 0
has_answer_correct = 0

label_pred_0_0 = 0
label_pred_0_1 = 0
label_pred_1_0 = 0
label_pred_1_1 = 0

for article in dev['data']: 
        for paragraph in article['paragraphs']:
            for q in paragraph['qas']:
                qid = q['id']

                if q['is_impossible']:
                    no_answer += 1
                    if pred[qid] == "":
                        no_answer_correct += 1
                        label_pred_0_0 += 1
                    else:
                        label_pred_0_1 += 1
                else:
                    has_answer += 1
                    if pred[qid] == "":
                        label_pred_1_0 += 1
                    else:
                        label_pred_1_1 += 1
                    for anw in q['answers']:
                        #print ("qid: %s, Label: %s, Pred: %s" % (qid, anw['text'], pred[qid])) 
                        if pred[qid] == anw['text']:
                            has_answer_correct += 1 
                            break 

print("label_pred_0_0: %d" % label_pred_0_0)
print("label_pred_0_1: %d" % label_pred_0_1)
print("label_pred_1_0: %d" % label_pred_1_0)
print("label_pred_1_1: %d" % label_pred_1_1)
print("no_answer: %d, no_answer_correct: %d (%f), has_answer: %d, has_answer_correct: %d (%f)" %
      (no_answer, no_answer_correct, no_answer_correct / no_answer, has_answer, has_answer_correct, has_answer_correct / has_answer))
                

label_pred_0_0: 54853
label_pred_0_1: 81
label_pred_1_0: 0
label_pred_1_1: 6291
no_answer: 54934, no_answer_correct: 54853 (0.998526), has_answer: 6291, has_answer_correct: 4912 (0.780798)


In [0]:
def GetQuestionContext(filename, qid):
    with open(filename) as f:
        d = json.load(f)  
    for article in d['data']: 
        for paragraph in article['paragraphs']:
            for q in paragraph['qas']:
                if qid == q['id']:
                    print(json.dumps(q))
                    print(paragraph['context'])
                    break

GetQuestionContext("./dev_sentence_json.json", "571142b3a58dae1900cd6d5c_387")
GetQuestionContext("./dev-v2.0.json", "571142b3a58dae1900cd6d5c")


{"question": "Above what horsepower are steam turbines usually more efficient than steam engines that use reciprocating pistons?", "id": "571142b3a58dae1900cd6d5c_387", "answers": [{"text": "several hundred", "answer_start": 108}, {"text": "several hundred horsepower", "answer_start": 108}, {"text": "several hundred", "answer_start": 108}], "is_impossible": false}
Steam turbines are generally more efficient than reciprocating piston type steam engines (for outputs above several hundred horsepower), have fewer moving parts, and provide rotary power directly instead of through a connecting rod system or similar means. ...
{"question": "Above what horsepower are steam turbines usually more efficient than steam engines that use reciprocating pistons?", "id": "571142b3a58dae1900cd6d5c", "answers": [{"text": "several hundred", "answer_start": 238}, {"text": "several hundred horsepower", "answer_start": 238}, {"text": "several hundred", "answer_start": 238}], "is_impossible": false}
The final

### **8.2 Aggregate Back To Context Level** 

In [5]:
cd $DRIVE_OUTPUT_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/model_output


In [8]:
import json
with open("./sentence_predictions.json") as f:
    pred = json.load(f)

# Json object of new prediction.
new_pred = {}

qid2answers = {}
for key in pred:
    text = pred[key]
    # Sentence level question id is in the format of "originalId_sentenceStartIndex"
    # This is to recover the original question id.
    new_key = key.split('_')[0]
    
    # As long as one sentence predicts an answer, there will be an answer at context level.
    # When multiple sentences predict answer, we take the one from last sentence.
    if new_key not in new_pred:
        new_pred[new_key] = ""
        qid2answers[new_key] = 0
    if text != "":
        qid2answers[new_key] += 1
        new_pred[new_key] = text

# Get some stats
has_answer = 0
no_answer = 0
multiple_answers = 0
line = 0
for x in new_pred:
    line += 1
    if new_pred[x] != "":
        has_answer += 1
    else:
        no_answer += 1
    if qid2answers[x] > 1:
        multiple_answers += 1    

print("Has_answer: %d, no_answer: %d, total: %d, line: %d" % (has_answer, no_answer, has_answer + no_answer, line))
print("Multiple answers: %d" % multiple_answers)
with open("./context_predictions.json", "w") as f:
    f.write(json.dumps(new_pred))

Has_answer: 5841, no_answer: 6032, total: 11873, line: 11873
Multiple answers: 491


### **8.3 Context Level Evaluation (Final Eval)** 

In [0]:
cd $SQUAD_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/squad_2


In [0]:
!python ./evaluate-v2.0.py  ./dev-v2.0.json $DRIVE_OUTPUT_DIR/context_predictions.json

{
  "exact": 90.38996041438558,
  "f1": 94.13821112727037,
  "total": 11873,
  "HasAns_exact": 81.39338731443995,
  "HasAns_f1": 88.9006377722813,
  "HasAns_total": 5928,
  "NoAns_exact": 99.36080740117745,
  "NoAns_f1": 99.36080740117745,
  "NoAns_total": 5945
}


In [0]:
# For testing / debugging purpose only.
import json
with open("../model_output/context_predictions.json") as f:
    pred = json.load(f)

with open("./dev-v2.0.json") as f:
    dev = json.load(f)
 
no_answer = 0
no_answer_correct = 0
has_answer = 0
has_answer_correct = 0

label_pred_0_0 = 0
label_pred_0_1 = 0
label_pred_1_0 = 0
label_pred_1_1 = 0

for article in dev['data']: 
        for paragraph in article['paragraphs']:
            for q in paragraph['qas']:
                qid = q['id']

                if q['is_impossible']:
                    no_answer += 1
                    if pred[qid] == "":
                        no_answer_correct += 1
                        label_pred_0_0 += 1
                    else:
                        label_pred_0_1 += 1
                else:
                    has_answer += 1
                    if pred[qid] == "":
                        label_pred_1_0 += 1
                    else:
                        label_pred_1_1 += 1
                    for anw in q['answers']:
                        #print ("qid: %s, Label: %s, Pred: %s" % (qid, anw['text'], pred[qid])) 
                        if pred[qid] == anw['text']:
                            has_answer_correct += 1 
                            break 

print("label_pred_0_0: %d" % label_pred_0_0)
print("label_pred_0_1: %d" % label_pred_0_1)
print("label_pred_1_0: %d" % label_pred_1_0)
print("label_pred_1_1: %d" % label_pred_1_1)
print("binary_accuracy: %f" % ((label_pred_1_1 + label_pred_0_0) / (label_pred_0_0 + label_pred_0_1 + label_pred_1_0 + label_pred_1_1)))
print("no_answer: %d, no_answer_correct: %d (%f), has_answer: %d, has_answer_correct: %d (%f)" %
      (no_answer, no_answer_correct, no_answer_correct / no_answer, has_answer, has_answer_correct, has_answer_correct / has_answer))

label_pred_0_0: 5907
label_pred_0_1: 38
label_pred_1_0: 125
label_pred_1_1: 5803
binary_accuracy: 0.986271
no_answer: 5945, no_answer_correct: 5907 (0.993608), has_answer: 5928, has_answer_correct: 4651 (0.784582)


### **8.4 What if we use it on top of model trained with only answerable data?**

In [0]:
!gsutil cp gs://bk_266/bert_output_m2/predictions.json $DRIVE_OUTPUT_DIR/HasAns_predictions.json

In [0]:
!ls -l ../model_output/

total 657881
-rw------- 1 root root    505117 Mar 30 04:04 context_predictions.json
-rw------- 1 root root  83460659 Mar 30 00:57 eval.tf_record
-rw------- 1 root root    337878 Apr  2 06:04 HasAns_predictions.json
-rw------- 1 root root      1782 Mar 30 04:03 input_file.json
-rw------- 1 root root 289743893 Mar 24 22:44 m2_nbest_predictions.json
-rw------- 1 root root    500654 Mar 24 23:30 m2_predictions_context.json
-rw------- 1 root root    428539 Mar 24 23:16 m2_predictions_context.line
-rw------- 1 root root   2573942 Mar 24 22:44 m2_predictions.json
-rw------- 1 root root   1960809 Mar 24 23:15 m2_predictions.line
-rw------- 1 root root    505117 Apr  2 06:02 merged_predictions.json
-rw------- 1 root root      1822 Mar 29 23:36 nbest_predictions.json
-rw------- 1 root root       165 Mar 29 23:36 predictions.json
-rw------- 1 root root 291055366 Mar 30 04:04 sentence_nbest_predictions.json
-rw------- 1 root root   2591230 Mar 30 04:04 sentence_predictions.json


In [0]:
# Consolidate the two predictions
# - If sentence-level model predicts no answer, then output ""
# - If sentence-level model predicts any answer, replace it with the answer
#   predicted by HasAns model, which is trained with answerable questions.
import json
import copy
with open("../model_output/context_predictions.json") as f:
    model_1 = json.load(f)

with open("../model_output/HasAns_predictions.json") as f:
    model_2 = json.load(f)

for key in model_1:
    if model_1[key] != "" and key in model_2: 
        if model_1[key] != model_2[key]:
            print("Replacing: %s vs %s" % (model_1[key], model_2[key]))
            model_1[key] = model_2[key]
        #else:
        #    print("Key not in m2: %s" % key)

with open("../model_output/merged_predictions.json", "w") as f:
    f.write(json.dumps(model_1))

Replacing: The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy vs Christian piety
Replacing: Christianity vs Catholicism (Christianity
Replacing: Alexius Komnenos vs Byzantine general Alexius Komnenos
Replacing: Armenians vs Turkish
Replacing: Duke Richard II of Normandy vs Duke Richard II of Normandy, and King Ethelred II of England
Replacing: Duke Richard II of Normandy vs Richard II of Normandy
Replacing: Duke William II vs Duke William II of Normandy
Replacing: Ireland vs Irish
Replacing: Wales vs the Marches
Replacing: 1064 vs 1018
Replacing: Isaac vs Richard the Lion-hearted
Replacing: Maciot de Bethencourt vs Enrique Pérez de Guzmán, 2nd Count de Niebla
Replacing: in the Channel Islands vs Channel Islands
Replacing: rounded arches vs rounded
Replacing: Gothic vs Early Gothic
Replacing: mosaics vs sculptured fonts, capitals, and more importantly mosaics
Replacing: computational vs computational pro

In [0]:
!python ./evaluate-v2.0.py  ./dev-v2.0.json $DRIVE_OUTPUT_DIR/merged_predictions.json

{
  "exact": 90.17939863555968,
  "f1": 93.8704722271538,
  "total": 11873,
  "HasAns_exact": 80.81983805668017,
  "HasAns_f1": 88.21257030246284,
  "HasAns_total": 5928,
  "NoAns_exact": 99.51219512195122,
  "NoAns_f1": 99.51219512195122,
  "NoAns_total": 5945
}


In [0]:
!head ../model_output/m2_predictions.json

{
    "56ddde6b9a695914005b9628_167": "France",
    "56ddde6b9a695914005b9629_167": "10th and 11th centuries",
    "56ddde6b9a695914005b962a_167": "",
    "56ddde6b9a695914005b962b_167": "",
    "56ddde6b9a695914005b962c_167": "",
    "5ad39d53604f3c001a3fe8d1_167": "",
    "5ad39d53604f3c001a3fe8d2_167": "",
    "5ad39d53604f3c001a3fe8d3_167": "",
    "5ad39d53604f3c001a3fe8d4_167": "",


# **9. Appendix**


## 9.1 Evaluate sentence level TSV prediction at context level

In [18]:
cd $BASE_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project


In [17]:
!head ./dev_sentence_prediction_0_0_0.1.csv

In what country is Normandy located?	1	1	[6.4798148e-04 9.9935204e-01]
In what country is Normandy located?	0	0	[9.9992657e-01 7.3441617e-05]
In what country is Normandy located?	0	0	[9.9993670e-01 6.3307016e-05]
In what country is Normandy located?	0	0	[9.999174e-01 8.263814e-05]
When were the Normans in Normandy?	1	1	[8.143190e-04 9.991856e-01]
When were the Normans in Normandy?	0	0	[9.999366e-01 6.346813e-05]
When were the Normans in Normandy?	0	0	[9.9993289e-01 6.7093526e-05]
When were the Normans in Normandy?	0	0	[0.99600095 0.003999  ]
From which countries did the Norse originate?	0	0	[9.9993563e-01 6.4345375e-05]
From which countries did the Norse originate?	1	1	[8.7946595e-04 9.9912053e-01]


In [0]:
import csv
# Expected TSV sentence level prediciton output:
# question \t prediction \t label \t more-columns-not-used
# This actually also works for context level prediction. 

# NOTE: change the file accordingly.
# The first 3 columns has to be (separated by '\t'):
#   question, predition, label
dev_pred =  open('./dev_sentence_prediction_0_0_0.1.csv', 'r') 
dev_pred_csv = csv.reader(dev_pred, delimiter='\t')

# Sentence level stats
s_total = 0
s_correct = 0 
s_pred_label_0_0 = 0
s_pred_label_0_1 = 0
s_pred_label_1_0 = 0
s_pred_label_1_1 = 0 

# Context level prediction and label map.
q2pred = {}
q2label = {}
for row in dev_pred_csv: 
    q = row[0]
    pred = row[1]
    label = row[2]
    s_total += 1
    if pred == label:
        s_correct += 1
    if pred == '0' and label == '0':
        s_pred_label_0_0 += 1
    if pred == '0' and label == '1':
        s_pred_label_0_1 += 1
    if pred == '1' and label == '0':
        s_pred_label_1_0 += 1
    if pred == '1' and label == '1':
        s_pred_label_1_1 += 1

    # As long as one sentence predicted HasAns, the whole context will have answer.
    # Start from 0 and only let '1' to override it.
    if not q in q2pred:
        q2pred[q] = 0
    if pred == '1':
        q2pred[q] = 1

    # As long as one sentence has label=HasAns, the whole context will have label=HasAns.
    # Start from 0 and only let '1' to override it.
    if not q in q2label:
        q2label[q] = 0
    if label == '1':
        q2label[q] = 1

print("Sentence level stats:")
print("Total examples: %d, correct: %d, accuracy: %f" % (s_total, s_correct, s_correct / s_total))
print("s_pred_label_0_0: %d" % s_pred_label_0_0)
print("s_pred_label_0_1: %d" % s_pred_label_0_1)
print("s_pred_label_1_0: %d" % s_pred_label_1_0)
print("s_pred_label_1_1: %d" % s_pred_label_1_1)


# Context level stats
c_total = 0
c_correct = 0 
c_pred_label_0_0 = 0
c_pred_label_0_1 = 0
c_pred_label_1_0 = 0
c_pred_label_1_1 = 0 

# Now let's check context level.
for key in q2label:
    c_total += 1
    if not key in q2pred:
        print("Warning: question not found in prediction: %s, will treat it as prediction error" % key)
    label = q2label[key]
    pred = q2pred[key]
    if label == pred:
        c_correct += 1
    if pred == 0 and label == 0:
        c_pred_label_0_0 += 1
    if pred == 0 and label == 1:
        c_pred_label_0_1 += 1
    if pred == 1 and label == 0:
        c_pred_label_1_0 += 1
    if pred == 1 and label == 1:
        c_pred_label_1_1 += 1  

print("\nContext level stats:")
print("Total examples: %d, correct: %d, accuracy: %f" % (c_total, c_correct, c_correct / c_total))
print("c_pred_label_0_0: %d" % c_pred_label_0_0)
print("c_pred_label_0_1: %d" % c_pred_label_0_1)
print("c_pred_label_1_0: %d" % c_pred_label_1_0)
print("c_pred_label_1_1: %d" % c_pred_label_1_1)

Sentence level stats:
Total examples: 64248, correct: 57490, accuracy: 0.894814
s_pred_label_0_0: 52392
s_pred_label_0_1: 773
s_pred_label_1_0: 5985
s_pred_label_1_1: 5098

Context level stats:
Total examples: 11864, correct: 7147, accuracy: 0.602411
c_pred_label_0_0: 1853
c_pred_label_0_1: 575
c_pred_label_1_0: 4142
c_pred_label_1_1: 5294


## 9.2 Double the NoAns questions simply by replicating them

In [0]:
cd $SQUAD_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/squad_2


In [0]:
import json

with open("./train-v2.0.json") as f:
    data = json.load(f)

for article in data['data']:
    for paragraph in article['paragraphs']:
        # Pick out the NoAns questions.
        no_ans_q = []
        for q in paragraph['qas']:
            if q['is_impossible']:
                no_ans_q.append(q)

        # Then add another copy of them.
        for q in no_ans_q:
            paragraph['qas'].append(q)

with open("./train_double_NoAns.json", 'w') as f:
    f.write(json.dumps(data))          

In [0]:
GetStats("./train_double_NoAns.json")
# Filename: ./train_double_NoAns.json, articles: 442, paragraphs: 19035, questions: 173817, questions_with_answer:86821, questons_no_answer: 86996, paragraphs without answerable questions: 153

Filename: ./train_double_NoAns.json, articles: 442, paragraphs: 19035, questions: 173817, questions_with_answer:86821, questons_no_answer: 86996, paragraphs without answerable questions: 153


## 9.3 Context-level Prediciton VS. Sentence-level Prediction

In [20]:
cd $SQUAD_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/squad_2


In [21]:
#!head ../model_output/context_predictions.json
!tail -10 ../model_output/context_model_predictions.json

    "5737aafd1c456719005744fb": "pound-force",
    "5737aafd1c456719005744fc": "kilopond",
    "5737aafd1c456719005744fd": "metric slug",
    "5737aafd1c456719005744fe": "",
    "5737aafd1c456719005744ff": "",
    "5ad28ad0d7d075001a4299cc": "",
    "5ad28ad0d7d075001a4299cd": "",
    "5ad28ad0d7d075001a4299ce": "",
    "5ad28ad0d7d075001a4299cf": ""
}


In [0]:
!python ./evaluate-v2.0.py  ./dev-v2.0.json ../model_output/context_predictions.json

{
  "exact": 90.38996041438558,
  "f1": 94.13821112727037,
  "total": 11873,
  "HasAns_exact": 81.39338731443995,
  "HasAns_f1": 88.9006377722813,
  "HasAns_total": 5928,
  "NoAns_exact": 99.36080740117745,
  "NoAns_f1": 99.36080740117745,
  "NoAns_total": 5945
}


In [0]:
# Normalization function copied from official SQuAD eval script.
# https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/
import argparse
import collections
import json
import numpy as np
import os
import re
import string
import sys

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

In [0]:
import json

# Original DEV set with labels 
with open("./dev-v2.0.json") as f:
    dev = json.load(f)

# The answer (context level) predicted by sentence-level model.
with open("../model_output/context_predictions.json") as f:
    sentence_model = json.load(f)

# The answer predictedby context-level model.
with open("../model_output/context_model_predictions.json") as f:
    context_model = json.load(f)   

# Files to store the DEV questions the two models predicted different:
# 1. HasAns questions, context-model got it right, but sentenc model is wrong.
# 2. HasAns questions, context-model got it wrong, but sentence model is right.
# 3. NoAns questions, context-model got it right, but sentence model is wrong.
# 4. NoAns questions, context-model got it wrong, but sentence model is right. 
# 5. Both context and sentence model got it wrong.
HasAns_context_correct_f = open("../model_output/HasAns_context_correct.tsv", "w")
HasAns_sentence_correct_f = open("../model_output/HasAns_sentence_correct.tsv", "w")
NoAns_context_correct_f = open("../model_output/NoAns_context_correct.tsv", "w")
NoAns_sentence_correct_f = open("../model_output/NoAns_sentence_correct.tsv", "w")
HasAns_both_wrong_f = open("../model_output/HasAns_both_wrong.tsv", "w")
NoAns_both_wrong_f = open("../model_output/NoAns_both_wrong.tsv", "w")

context_pred_HasAns = 0
context_pred_NoAns = 0
sentence_pred_HasAns = 0
sentence_pred_NoAns = 0

for article in dev['data']: 
    for paragraph in article['paragraphs']:
        for question in paragraph['qas']:
            qid = question['id']

            # Get sentence model prediction.
            sentence_pred = ""
            if not qid in sentence_model:
                print("Warning: id not exist in sentence_model: %s" % qid)
            else:
                sentence_pred = normalize_answer(sentence_model[qid])
            
            if sentence_pred =="":
                sentence_pred_NoAns += 1
            else:
                sentence_pred_HasAns += 1

            # Get context model prediction.
            context_pred = ""
            if not qid in context_model:
                print("Warning: id not exist in context_model: %s" % qid)
            else:
                context_pred = normalize_answer(context_model[qid])

            if context_pred == "":
                context_pred_NoAns += 1
            else:
                context_pred_HasAns +=1

            # Output the qid, setence-prediction, context-prediction, question, context, answer.
            # Add 'P:" prefix to predictions so that we can see empty predictions easier.
            out_record = ("QID:" + qid + "\t" + "SP:" + sentence_pred + "\t" + "CP:" + context_pred + "\t"
                             + "Q:" + question['question'].replace("\n", " ") + "\t" + "C:" + paragraph['context'].replace("\n", " ") + "\t" + "A:" + str(question['answers']) + "\n")
            # Question not answerable.
            if question['is_impossible']:
                if sentence_pred == "" and context_pred == "":
                    # both correct, no output
                    pass
                elif sentence_pred == "" and context_pred != "":
                    # sentence model correct, context model wrong.
                    NoAns_sentence_correct_f.write(out_record)
                elif sentence_pred != "" and context_pred == "":    
                    # sentence model wrong, context model right.
                    NoAns_context_correct_f.write(out_record)
                else:  # both are wrong
                    NoAns_both_wrong_f.write(out_record)
            else:
                # The question has answer, we'll need to check all candidate answers.
                sentence_correct = False
                context_correct = False
                for ans in question['answers']:
                    ans_norm = normalize_answer(ans['text'])
                    if ans_norm == sentence_pred:
                        sentence_correct = True
                    if ans_norm == context_pred:
                        context_correct = True

                if sentence_correct and context_correct:
                    pass
                elif sentence_correct and not context_correct:
                    HasAns_sentence_correct_f.write(out_record)
                elif not sentence_correct and context_correct:
                    HasAns_context_correct_f.write(out_record) 
                else: # both are wrong
                    HasAns_both_wrong_f.write(out_record) 

HasAns_context_correct_f.close()
HasAns_sentence_correct_f.close()
NoAns_context_correct_f.close()
NoAns_sentence_correct_f.close()
HasAns_both_wrong_f.close()
NoAns_both_wrong_f.close()

print("Context model pred HasAns: %d, NoAns:%d" % (context_pred_HasAns, context_pred_NoAns))
print("Sentence model pred HasAns: %d, NoAns:%d" % (sentence_pred_HasAns, sentence_pred_NoAns))

Context model pred HasAns: 6708, NoAns:5165
Sentence model pred HasAns: 5841, NoAns:6032


In [0]:
%ls

1e3_dev_sentence_prediction_0_10000.csv   model_1_output.tsv
1e4_dev_sentence_prediction_0_10000.csv   model_1_output_with_header.csv
1e5_dev_sentence_prediction_0_10000.csv   old/
dev_answerable.json                       train_answerable.json
dev_context.tsv                           train_context_json_190k.tsv
dev_mrpc.tsv                              train_context.tsv
dev_sentence_json.json                    train_double_NoAns.json
dev_sentence_json.line                    train_mrpc.tsv
dev_sentence_json.tsv                     train_sentence_json_100k.json
dev_sentence_mrpc_2.tsv                   train_sentence_json_190k.json
dev_sentence_mrpc.tsv                     train_sentence_json_190k.line
dev_sentence_prediction_0_0_0.0001.csv    train_sentence_json_190k.tsv
dev_sentence_prediction_0_0_0_01.csv      train_sentence_json_190k_tt.json
dev_sentence_prediction_0_0_0.01.csv      train_sentence_json_3_1_130k.json
dev-v2.0.json                             train_sentence_json_3_

In [0]:
!mv ../model_output/drive-download-20200410T063033Z-001 ../model_output/rs

In [0]:
!echo "HasAns, Context correct, Sentence wrong------"
!wc -l ../model_output/rs/HasAns_context_correct.tsv 
!head  ../model_output/rs/HasAns_context_correct.tsv


HasAns, Context correct, Sentence wrong------
436 ../model_output/rs/HasAns_context_correct.tsv
QID:56dde27d9a695914005b9651	SP:Catholicism (Christianity	CP:Catholicism	Q:What was the Norman religion?	C:The descendants of Rollo's Vikings and their Frankish wives would replace the Norse religion and Old Norse language with Catholicism (Christianity) and the Gallo-Romance language of the local people, blending their maternal Frankish heritage with Old Norse traditions and customs to synthesize a unique "Norman" culture in the north of France. The Norman language was forged by the adoption of the indigenous langue d'oïl branch of Romance by a Norse-speaking ruling class, and it developed into the regional language that survives today.	A:[{'text': 'Catholicism', 'answer_start': 121}, {'text': 'Catholicism', 'answer_start': 121}, {'text': 'Catholicism', 'answer_start': 121}]
QID:56de0ffd4396321400ee258f	SP:Byzantine general Alexius Komnenos	CP:Alexius Komnenos	Q:Who ruined Roussel de Baille

In [0]:
!cat ../model_output/HasAns_context_correct.tsv

cat: ../model_output/HasAns_context_correct.tsv: No such file or directory


In [0]:
!echo "HasAns, Context wrong, Sentence correct------"
!wc -l ../model_output/rs/HasAns_sentence_correct.tsv
!head  ../model_output/rs/HasAns_sentence_correct.tsv


HasAns, Context wrong, Sentence correct------
712 ../model_output/rs/HasAns_sentence_correct.tsv
QID:56ddde6b9a695914005b962a	SP:Denmark, Iceland and Norway	CP:	Q:From which countries did the Norse originate?	C:The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.	A:[{'text': 'Denmark, Iceland and Norway', 'an

In [0]:
!echo "HasAns, both wrong------"
!wc -l ../model_output/HasAns_both_wrong.tsv 
!head  ../model_output/HasAns_both_wrong.tsv 

HasAns, both wrong------
695 ../model_output/HasAns_both_wrong.tsv
QID:56dde0ba66d3e219004dad76	SP:king charles iii of west francia	CP:king charles iii of west francia	Q:Who did Rollo sign the treaty of Saint-Clair-sur-Epte with?	C:In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more permanent encampments that included local women and personal property. The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was situated in the former Frankish kingdom of Neustria. The treaty offered Rollo and his men the French lands between the river Epte and the Atlantic coast in exchange for their protection against further Viking incursions. The area corresponded to the northern part of present-day Upper Normandy down to the river Seine, but the Duchy would eventually extend west beyond 

In [0]:
!echo "NoAns, Context correct, Sentence wrong------"
!wc -l ../model_output/rs/NoAns_context_correct.tsv 
!head ../model_output/rs/NoAns_context_correct.tsv


NoAns, Context correct, Sentence wrong------
31 ../model_output/rs/NoAns_context_correct.tsv
QID:5a2c07d5bfd06b001a5ae983	SP:Isleworth	CP:	Q:Where did the headquarters relocate from?	C:Sky UK Limited (formerly British Sky Broadcasting or BSkyB) is a British telecommunications company which serves the United Kingdom. Sky provides television and broadband internet services and fixed line telephone services to consumers and businesses in the United Kingdom. It is the UK's largest pay-TV broadcaster with 11 million customers as of 2015. It was the UK's most popular digital TV service until it was overtaken by Freeview in April 2007. Its corporate headquarters are based in Isleworth.	A:[]
QID:5a2c0fb8bfd06b001a5ae9b9	SP:branded Open	CP:	Q:When was BSkyB's digital service unofficially launched?	C:BSkyB's digital service was officially launched on 1 October 1998 under the name Sky Digital, although small-scale tests were carried out before then. At this time the use of the Sky Digital brand m

In [0]:
ls

sample_data/


In [0]:
!cat ../model_output/rs/NoAns_context_correct.tsv

QID:5a2c07d5bfd06b001a5ae983	SP:Isleworth	CP:	Q:Where did the headquarters relocate from?	C:Sky UK Limited (formerly British Sky Broadcasting or BSkyB) is a British telecommunications company which serves the United Kingdom. Sky provides television and broadband internet services and fixed line telephone services to consumers and businesses in the United Kingdom. It is the UK's largest pay-TV broadcaster with 11 million customers as of 2015. It was the UK's most popular digital TV service until it was overtaken by Freeview in April 2007. Its corporate headquarters are based in Isleworth.	A:[]
QID:5a2c0fb8bfd06b001a5ae9b9	SP:branded Open	CP:	Q:When was BSkyB's digital service unofficially launched?	C:BSkyB's digital service was officially launched on 1 October 1998 under the name Sky Digital, although small-scale tests were carried out before then. At this time the use of the Sky Digital brand made an important distinction between the new service and Sky's analogue services. Key selling

In [0]:
!echo "NoAns, Context wrong, Sentence correct------"
!wc -l ../model_output/rs/NoAns_sentence_correct.tsv 
!head  ../model_output/rs/NoAns_sentence_correct.tsv 

NoAns, Context wrong, Sentence correct------
1368 ../model_output/rs/NoAns_sentence_correct.tsv
QID:5ad39d53604f3c001a3fe8d3	SP:	CP:The Normans	Q:Who did King Charles III swear fealty to?	C:The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.	A:[]
QID:5ad3a266604f3c001a3fea2a	SP:	CP:Richard I	Q:Who ruled the 

In [0]:
!grep '366,273' ../model_output/rs/NoAns_sentence_correct.tsv 

QID:5a1c8f54b4fb5d00187146d8	SP:	CP:2010	Q:What year was 366,273 people in Jacksonville?	C:As of 2010[update], there were 366,273 households out of which 11.8% were vacant. 23.9% of households had children under the age of 18 living with them, 43.8% were married couples, 15.2% had a female householder with no husband present, and 36.4% were non-families. 29.7% of all households were made up of individuals and 7.9% had someone living alone who was 65 years of age or older. The average household size was 2.55 and the average family size was 3.21. In the city, the population was spread out with 23.9% under the age of 18, 10.5% from 18 to 24, 28.5% from 25 to 44, 26.2% from 45 to 64, and 10.9% who were 65 years of age or older. The median age was 35.5 years. For every 100 females there were 94.1 males. For every 100 females age 18 and over, there were 91.3 males.	A:[]
QID:5a1c8f54b4fb5d00187146d9	SP:	CP:23.9%	Q:How many of Jacksonville city's residents are older than eighteen?	C:As of 2010

In [0]:
!echo "NoAns, both wrong------"
!wc -l ../model_output/NoAns_both_wrong.tsv 
!head  ../model_output/NoAns_both_wrong.tsv 

NoAns, both wrong------
7 ../model_output/NoAns_both_wrong.tsv
QID:5a38b07ca4b263001a8c189f	SP:hundred times price youve paid us	CP:more for oil lets say ten times more	Q:What did Syria decide that the price of oil should be after the attack?	C:On October 6, 1973, Syria and Egypt, with support from other Arab nations, launched a surprise attack on Israel, on Yom Kippur. This renewal of hostilities in the Arab–Israeli conflict released the underlying economic pressure on oil prices. At the time, Iran was the world's second-largest oil exporter and a close US ally. Weeks later, the Shah of Iran said in an interview: "Of course [the price of oil] is going to rise... Certainly! And how!... You've [Western nations] increased the price of the wheat you sell us by 300 percent, and the same for sugar and cement... You buy our crude oil and sell it back to us, refined as petrochemicals, at a hundred times the price you've paid us... It's only fair that, from now on, you should pay more for oil.

## 9.4 Average paragrph length

In [0]:
cd $SQUAD_DATA_DIR

/content/drive/My Drive/Colab Notebooks/w266_project/squad_2


In [0]:
../model_output/HasAns_context_correct.tsv

In [0]:
import json
import csv
import re
 
noans_sentence =  open('./NoAns_context_correct.tsv', 'r') 
noans_sentence_csv = csv.reader(noans_sentence, delimiter='\t')
context_length = 0
count = 0
for row in noans_sentence_csv: 
    count += 1
    qid = row[0]
    SP = row[1]
    CP = row[2]
    question = row[3]
    context = row[4]
    answer = row[5]
    context_length +=len(context)
    print(qid + '\t' + SP + '\t' + CP + '\t' + question + '\t'+ context + '\t' + answer + '\t' )

avg_context_length = context_length/count

print('context avg length:', avg_context_length)


QID:5a2c07d5bfd06b001a5ae983	SP:Isleworth	CP:	Q:Where did the headquarters relocate from?	C:Sky UK Limited (formerly British Sky Broadcasting or BSkyB) is a British telecommunications company which serves the United Kingdom. Sky provides television and broadband internet services and fixed line telephone services to consumers and businesses in the United Kingdom. It is the UK's largest pay-TV broadcaster with 11 million customers as of 2015. It was the UK's most popular digital TV service until it was overtaken by Freeview in April 2007. Its corporate headquarters are based in Isleworth.	A:[]	
QID:5a2c0fb8bfd06b001a5ae9b9	SP:branded Open	CP:	Q:When was BSkyB's digital service unofficially launched?	C:BSkyB's digital service was officially launched on 1 October 1998 under the name Sky Digital, although small-scale tests were carried out before then. At this time the use of the Sky Digital brand made an important distinction between the new service and Sky's analogue services. Key sellin

In [0]:
import json
import csv
import re
 
noans_sentence =  open('./NoAns_sentence_correct.tsv', 'r') 
noans_sentence_csv = csv.reader(noans_sentence, delimiter='\t')
context_length = 0
count = 0
for row in noans_sentence_csv: 
    count += 1
    qid = row[0]
    SP = row[1]
    CP = row[2]
    question = row[3]
    context = row[4]
    answer = row[5]
    context_length +=len(context)
    print(qid + '\t' + SP + '\t' + CP + '\t' + question + '\t'+ context + '\t' + answer + '\t' )

avg_context_length = context_length/count

print('context avg length:', avg_context_length)
 

QID:5ad39d53604f3c001a3fe8d3	SP:	CP:The Normans	Q:Who did King Charles III swear fealty to?	C:The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.	A:[]	
QID:5ad3a266604f3c001a3fea2a	SP:	CP:Richard I	Q:Who ruled the country of Normandy?	C:The Norman dynasty had a major political, cultural and military impact o